# Parameters

## Imports

In [86]:
%load_ext autoreload
%autoreload 2

import dolfin
import numpy
import math
import dolfin_mech     as dmech
import micro_poro_structure_generator as gen

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Defining geometry

In [87]:
seeds_filename = "ch4.dat"
mesh_filebasename = "ch4-mesh"

domain_y = 0.1 * 0.8
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.012 * 0.8

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = - 0.023 * 0.8,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)
dV = dolfin.Measure("dx",domain=mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])

V = (xmax - xmin)*(ymax - ymin)
VS0 = dolfin.assemble(dolfin.Constant(1) * dV)
Vf0 = V - VS0

[[0.02309401 0.02      ]
 [0.         0.06      ]
 [0.06928203 0.02      ]
 [0.04618802 0.06      ]]
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 122 (Line)
Info    : [ 10%] Meshing curve 123 (Line)
Info    : [ 10%] Meshing curve 124 (Line)
Info    : [ 20%] Meshing curve 148 (Line)
Info    : [ 20%] Meshing curve 149 (Line)
Info    : [ 30%] Meshing curve 150 (Line)
Info    : [ 30%] Meshing curve 151 (Line)
Info    : [ 30%] Meshing curve 152 (Line)
Info    : [ 40%] Meshing curve 153 (Line)
Info    : [ 40%] Meshing curve 154 (Line)
Info    : [ 50%] Meshing curve 155 (Line)
Info    : [ 50%] Meshing curve 156 (Line)
Info    : [ 50%] Meshing curve 157 (Line)
Info    : [ 60%] Meshing curve 158 (Line)
Info    : [ 60%] Meshing curve 159 (Line)
Info    : [ 70%] Meshing curve 160 (Line)
Info    : [ 70%] Meshing curve 161 (Line)
Info    : [ 80%] Meshing curve 162 (Line)
Info    : [ 80%] Meshing curve 

/Users/tayebi/miniconda3/envs/all/lib/python3.8/site-packages/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:44: RuntimeWarning: divide by zero encountered in double_scalars
  m = (Q[1] - P[1])/(Q[0] - P[0])
/Users/tayebi/miniconda3/envs/all/lib/python3.8/site-packages/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:61: RuntimeWarning: invalid value encountered in double_scalars
  x = (c2 - c1)/(m1 - m2)
/Users/tayebi/miniconda3/envs/all/lib/python3.8/site-packages/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:62: RuntimeWarning: invalid value encountered in double_scalars
  y = (m2*c1 - m1*c2)/(m2 - m1)
/Users/tayebi/miniconda3/envs/all/lib/python3.8/site-packages/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:45: RuntimeWarning: invalid value encountered in double_scalars
  c = P[1] - m * P[0]


Info    : Running '/Users/tayebi/miniconda3/envs/all/bin/gmsh -2 -o ch4-mesh.msh -format msh22 ch4-mesh.msh' [Gmsh 4.9.0, 1 node, max. 1 thread]
Info    : Started on Mon May 20 21:52:21 2024
Info    : Reading 'ch4-mesh.msh'...
Info    : 55 entities
Info    : 550 nodes
Info    : 916 elements
Info    : Done reading 'ch4-mesh.msh'
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 5.1554e-05s, CPU 0.000245s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 2.891e-05s, CPU 0.00012s)
Info    : 562 nodes 946 elements
Info    : Writing 'ch4-mesh.msh'...
Info    : Done writing 'ch4-mesh.msh'
Info    : Stopped on Mon May 20 21:52:21 2024 (From start: Wall 0.00774503s, CPU 0.267339s)
Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Expecting 550 vertices
Found all vertices
Expecting 916 cells
Found all cells
Conversion done
Phis0=0.37249219381653026
Phif0=0.6275078061834698


In [88]:
mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)
dV = dolfin.Measure("dx",domain=mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])

V = (xmax - xmin)*(ymax - ymin)
VS0 = dolfin.assemble(dolfin.Constant(1) * dV)
Vf0 = V - VS0

## Loading 

In [89]:
qois_filename = "Fig5-qois.dat"
res_basename = "Fig5"

pf= 1
load_params = {}
load_params["pf"] = pf
load_params["sigma_bar_00"] = 0.0
load_params["sigma_bar_11"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["gamma"] = 15e-3

## Identifier function

In [90]:
params_dimless = [0, 0,  0, 0]
params_initial = [0.12414156917794349, 0.011853212744289299, 0.6282749906141952, 3.124581670582977] 

par = [params_initial[0]*(1 + params_dimless[0]), params_initial[1]*(1 + params_dimless[1]), params_initial[2]*(1 + params_dimless[2]), params_initial[3]*(1 + params_dimless[3])]
print(par)
# graph_printer(smith_gamma_0_inf, par)

mat_params = {"model":"exponential", "parameters":{"beta1":par[0], "beta2":par[1], "beta3":par[2], "alpha":par[3]}} 


problem = dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.005, "dt_max":0.1},
    res_basename=res_basename,
    write_qois_limited_precision=False,
    verbose=1
)

[0.12414156917794349, 0.011853212744289299, 0.6282749906141952, 3.124581670582977]


/Users/tayebi/miniconda3/envs/all/lib/python3.8/site-packages/ufl/exproperators.py:336: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if arg in ("+", "-"):


In [91]:
for operator in problem.operators: 
    if hasattr(operator, "material"):
        material = operator.material
        break

In [92]:
# Macroscopic strain
print("U_bar_x:" +str(problem.get_macroscopic_stretch_subsol().func.vector().get_local()[0]))
print("U_bar_y:" +str(problem.get_macroscopic_stretch_subsol().func.vector().get_local()[3]))
print("U_bar_xy:" +str(problem.get_macroscopic_stretch_subsol().func.vector().get_local()[1]))

# Kinematics
U_bar = problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))
F_bar = U_bar + numpy.eye(2)
C_bar = F_bar.T * F_bar
E_bar = 0.5*(C_bar - numpy.eye(2))
J_bar = numpy.linalg.det(F_bar)
FmTN = dolfin.dot(dolfin.inv(problem.kinematics.F).T, problem.mesh_normals)

print("J: "+str(J_bar))


# Poromechanics propoerties
T = dolfin.sqrt(dolfin.inner(FmTN, FmTN))
S = dolfin.assemble(T * problem.kinematics.J * problem.dS(0))
S0 = dolfin.assemble(dolfin.Constant(1)*problem.dS(0))


print("S:" +str(S))
print("S*: " + str(S/S0))
print("IC: "+str(numpy.trace(C_bar)))
print("IC_bar: "+str(numpy.trace(C_bar*J_bar**(-1))))

# Volumes
v = J_bar * V
print ("v:", v)

vs = dolfin.assemble(problem.kinematics.J * problem.dV)
print ("Vs0:", VS0)
print ("vs:", vs)
print ("Phi_s", vs/V)
vf = v - vs
print("v - vs:" +str(v - vs))
print("vf:" +str(vf))
print("porosity:"+str((V - VS0)/V))

# Macroscopic stress
print("sigma_tot_xx: " +str((dolfin.assemble(material.sigma[0,0] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[0,0])/v))
print("sigma_tot_xy: " +str((dolfin.assemble(material.sigma[0,1] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[0,1])/v))
print("sigma_tot_yx: " +str((dolfin.assemble(material.sigma[1,0] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[1,0])/v))
print("sigma_tot_yy: " +str((dolfin.assemble(material.sigma[1,1] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[1,1])/v))

print("sigma_s_xx: " +str((dolfin.assemble(material.sigma[0,0] * problem.kinematics.J * dV))/v))
print("sigma_s_xy: " +str((dolfin.assemble(material.sigma[0,1] * problem.kinematics.J * dV))/v))
print("sigma_s_xy: " +str((dolfin.assemble(material.sigma[1,0] * problem.kinematics.J * dV))/v))
print("sigma_s_yy: " +str((dolfin.assemble(material.sigma[1,1] * problem.kinematics.J * dV))/v))


print("sigma_f_xx: " +str((- vf * pf *dolfin.Identity(2)[0,0])/v))
print("sigma_f_xy: " +str((- vf * pf *dolfin.Identity(2)[0,1])/v))
print("sigma_f_xy: " +str((- vf * pf *dolfin.Identity(2)[1,0])/v))
print("sigma_f_yy: " +str((- vf * pf *dolfin.Identity(2)[1,1])/v))

#  numpy.linalg.det(problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))+numpy.eye(2))
# dolfin.det(dolfin.Identity(2) + dolfin.grad(problem.U_bar))

vf = v - vs

Vf0 = V - VS0

print("added fluid mass density: " +str((vf - Vf0)/V))

print("deformed porosity: " +str((vf)/v))

print("deformed surface: " +str(2 * math.pi * numpy.sqrt(vf/math.pi)))

U_bar_x:0.8260069081700966
U_bar_y:0.8223172162106626
U_bar_xy:-3.420909611788229e-05
J: 3.327563824507707
S:0.5351935324045042
S*: 2.111306631497334
IC: 6.655141265182694
IC_bar: 2.0000040919327167
v: 0.012295487166881192
Vs0: 0.001376374197574452
vs: 0.001432301203101195
Phi_s 0.3876278836739528
v - vs:0.010863185963779998
vf:0.010863185963779998
porosity:0.6275078061834698
sigma_tot_xx: -0.32671901207968024
sigma_tot_xy: -9.486765889047053e-06
sigma_tot_yx: -9.486765889043267e-06
sigma_tot_yy: -0.32619557984399394
sigma_s_xx: 0.5567909958068553
sigma_s_xy: -9.486765889047053e-06
sigma_s_xy: -9.486765889043267e-06
sigma_s_yy: 0.5573144280425415
sigma_f_xx: -0.8835100078865354
sigma_f_xy: 0
sigma_f_xy: 0
sigma_f_yy: -0.8835100078865354
added fluid mass density: 2.312428134650285
deformed porosity: 0.8835100078865354
deformed surface: 0.36947370795980056


In [93]:
problem.Vs0 

0.001376374197574452